In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import math
import cv2
import os
import imageio

from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline

def process_image(image):
    kernel_size = 7

    low_threshold = 50
    high_threshold = 200

    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(int(imshape[1]/2)-50, 330),(int(imshape[1]/2)+50, 330), (imshape[1],imshape[0])]], dtype=np.int32)

    rho = 1
    theta = np.pi/180
    threshold = 15
    min_line_len = 100
    max_line_gap = 150


    gray = grayscale(image)
    gb= gaussian_blur(gray,kernel_size)
    can = canny(gb, low_threshold,high_threshold)
    ri = region_of_interest(can,vertices)
    hl = hough_lines(ri, rho, theta, threshold, min_line_len, max_line_gap)
    wi = weighted_img(hl, image, 0.8, 1.0, 0)
  
    return wi

def grayscale(img):
    return cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img,kernel_size):
    return cv2.GaussianBlur(img,(kernel_size,kernel_size),0)

def region_of_interest(img,vertices):
    mask = np.zeros_like(img)
    if len(img.shape)>2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,)*channel_count
    else:
        ignore_mask_color = 255
    
    cv2.fillPoly(mask,vertices,ignore_mask_color)
    masked_image = cv2.bitwise_and(img,mask)
    return masked_image

def avarage(line):
    nl =[]
    p =0
    for i in range(len(line[0][0])):
        for j in range(len(line)):
            p = p + line[j][0][i]  
        nl.append(p/len(line))
        p=0
    return list(map(int,nl))
    
def draw_lines(img,lines,color=[255,0,0],thickness = 12):
    rg1 =[]
    rg2 =[]
    lf1 = []
    lf2  =[] 
    for line in lines:         
        for x1,y1,x2,y2 in line:          
            a = ((y2-y1)/(x2-x1))   
            
            #right
            if a>0:
                if x2< 500:
                    pass
                else:
                    rg1.append(line)                
            #left
            else:
                lf1.append(line)

    rg2 = avarage(rg1)
    a = ((rg2[3]-rg2[1])/(rg2[2]-rg2[0]))
    b = rg2[3]-a*rg2[2]
    rg2[0]=530
    rg2[2] = 960
    cv2.line(img,(rg2[0],int(rg2[0]*a+b)),(rg2[2],int(rg2[2]*a+b)),color,thickness)
    
    lf2 = avarage(lf1)
    a = ((lf2[3]-lf2[1])/(lf2[2]-lf2[0]))
    b = lf2[3]-a*lf2[2]
    lf2[2] = 430
    cv2.line(img,(0,lf2[1]-int(a*lf2[0])),(lf2[2],int(lf2[2]*a+b)),color,thickness)                

def hough_lines(img,rho,theta,threshold,min_line_len,max_line_gap):
    lines = cv2.HoughLinesP(img,rho,theta,threshold,np.array([]),minLineLength = min_line_len, maxLineGap = max_line_gap)
    line_img = np.zeros((img.shape[0],img.shape[1],3), dtype = np.uint8)
    draw_lines(line_img,lines)
    return line_img

def weighted_img(img, initial_img, alpha = 0.8, beta = 1. , gama = 0):
    return cv2.addWeighted(initial_img, alpha, img, beta, gama)

# image = mpimg.imread('test_images/whiteCarLaneSwitch.jpg')
# final = process_image(image)
# plt.imshow(final, cmap='gray')
# plt.show()

white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip('test_videos/solidWhiteRight.mp4')
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio = False)
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))


[MoviePy] >>>> Building video test_videos_output/12.mp4
[MoviePy] Writing video test_videos_output/12.mp4


100%|███████████████████████████████████████████████████████████████████████████████▋| 221/222 [00:02<00:00, 86.02it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/12.mp4 

Wall time: 2.86 s
